# Phase 4: Advanced Models & Ensembles

**Note:** This notebook is under reconstruction after MLflow integration.

For now, please use notebook 03 for baseline models.

Advanced models (LightGBM tuned, XGBoost, CatBoost, Ensemble) will be added here.

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'src'))

import pandas as pd
import numpy as np
import mlflow
from utils.mlflow_utils import setup_mlflow, log_dvc_data_version

experiment_id = setup_mlflow()
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")
print("Setup complete!")

## Status

This notebook is being reconstructed.

**Working:**
- Notebook 03: Baseline models with MLflow tracking
- DVC data versioning integration

**To be added:**
- LightGBM Tuned model
- XGBoost model  
- CatBoost model
- Weighted ensemble

Please use notebook 03 for now.